In [ ]:
######################
#####Figure S3C ######
######################
#read in CLIP samfile
#divide samfile by chromosome (using samtools)?

#parse through fasta file for each chromosome and compare with CLIP tags
#calculate shannon entropy scores 
#filter tags with high entropy scores

#question: calculate shannon entropy scores along each chromosome position or along tags???

#find the position with the highest entropy scores and re-align tags 


#LIN28A binding motif identified by CLIP-seq. 
#Sequences harboring a mutation were aligned with the mutated nucleotide centered at zero. 

In [1]:
from collections import defaultdict
import pysam

def readinSAM(chrNum):
    samfile = pysam.AlignmentFile("./LIN28A_CLIP.sorted.bam", "rb")
    cigarDic = defaultdict(str)
    seqDic = defaultdict(str)
    mapDic = defaultdict(int)
    reverseDic = defaultdict(bool)
    for read in samfile.fetch(chrNum):
        #if read.is_reverse == False:
        index = read.query_name
        cigarDic[index] = read.cigarstring
        seqDic[index] = read.query_sequence
        mapDic[index] = read.reference_start
        reverseDic[index] = read.is_reverse
    samfile.close()
    return cigarDic, seqDic, mapDic, reverseDic



In [2]:
#get fasta sequence for each chromosome
from Bio import SeqIO

def readinFasta(chrNum):
    for record in SeqIO.parse("./references/GRCm39.primary_assembly.genome.fa", "fasta"):
        if record.id == chrNum:
            refseq = str(record.seq)
    return refseq



In [4]:
from collections import defaultdict
import re
softclip = re.compile('[S](\d)*')
match = re.compile('(\d)*[M]')
deletion = re.compile('[D](\d)*')
insertion = re.compile('[I](\d)*')
forsplit = re.compile('(\d*[A-Z])')

def matchFrame (cigarDic, seqDic):
    newseqDic = defaultdict(str)
    for read in seqDic.keys():
        cigar = cigarDic[read]
        sequence = seqDic[read]
        newseq = ''
        count = 0
        cigarlist = forsplit.split(cigar)
        for i in cigarlist:
            if match.search(i) != None:
                matched =  int(i.rstrip('M'))
                newseq += sequence[count:count+matched]
                count += matched
            elif softclip.search(i) != None:
                clippedLen = int(i.rstrip('S'))
                count += clippedLen
            elif deletion.search(i) != None:
                deletedLen = int(i.rstrip('D'))
                newseq += 'N'*deletedLen
            elif insertion.search(i) != None:
                insertedLen = int(i.rstrip('I'))
                count += insertedLen
                sequence = sequence[:count]+sequence[count+insertedLen:]
                newseq += sequence
        newseqDic[read] = newseq
    return dict(sorted(newseqDic.items()))
    


In [5]:
from Bio.Seq import Seq
a = Seq('ACTG')
a.reverse_complement()
Seq('AAAA').reverse_complement()

Seq('TTTT')

In [6]:
#go through each frame-matched read sequence and compare with the ref sequence
#create numpy array for calculating frequencies
#A, T, C, G, N

#calculate shannon entropy at once?

import numpy as np
from collections import defaultdict
import datetime
from Bio.Seq import Seq

nuc_dict = {'A':0, 'C':1, 'G':2, 'T':3, 'N':4}

def seqcompare (chrseq, edited_seqDic, chr_mapDic, reverseDic):
    pos_nucfreq_arr = np.zeros((len(chrseq), 5), dtype=float)
    neg_nucfreq_arr = np.zeros((len(chrseq), 5), dtype=float)
    pos_depth_arr = np.zeros((len(chrseq), 1), dtype=int)
    neg_depth_arr = np.zeros((len(chrseq), 1), dtype=int)
    
    for count, read in enumerate(edited_seqDic.keys()):
        if count%1000000==0: 
            print(datetime.datetime.now())
        clip_seq = edited_seqDic[read]
        pos = chr_mapDic[read]
        for nuc in clip_seq:
            if reverseDic[read] == False:
                pos_depth_arr[pos,0] += 1
                if chrseq[pos] != nuc:
                    pos_nucfreq_arr[pos, nuc_dict[nuc]] += 1
                pos += 1
                if pos == len(chrseq):
                    break
            elif reverseDic[read] == True:
                neg_depth_arr[pos,0] += 1
                if chrseq[pos] != nuc:
                    neg_nucfreq_arr[pos, nuc_dict[nuc]] += 1
                pos += 1
                if pos == len(chrseq):
                    break
            
    return pos_nucfreq_arr, neg_nucfreq_arr, pos_depth_arr, neg_depth_arr
                
  

In [7]:
#calculate shannon entropy for each position

from collections import defaultdict
import numpy as np
from scipy.special import entr


def calcShannon (nuc_arr):
    with np.errstate(divide='ignore', invalid='ignore'):
        prob_arr = nuc_arr/(nuc_arr.sum(axis=1, keepdims=True)+10e-20)
    e_arr = entr(prob_arr).sum(axis=1, keepdims=True)
    return e_arr




In [8]:
#retrieve list of chromosomes

from Bio import SeqIO

def getChrlist():
    chr_list = []
    for record in SeqIO.parse("./references/GRCm39.primary_assembly.genome.fa", "fasta"):
        if record.id not in chr_list:
            chr_list.append(record.id)
    return chr_list

chr_list = getChrlist()

In [9]:
#filter each nucleotide position by its depth and entropy
def filter_index_list(entropy_arr, depth_arr):
    np.where(depth_arr > 50, entropy_arr, 0) #change entropy values in entropy_arr to 0 if smaller than 50
    filtered_ind = np.where(entropy_arr > 0.8) #index of entropy values bigger than 0.8
    filtered_entropy = np.take(entropy_arr, filtered_ind)[0,:] #get entropy values in accordance with filtered index values
    temp_idx = (0,0)
    temp_ind = 0
    temp_e = 0
    ind_list = list()
    e_list = list()
    for idx, ind in np.ndenumerate(filtered_ind[0]):
        #if ind == 48689527 or ind == 48689528:
         #   print('correct')
          #  print(filtered_entropy[idx])
        if ind-temp_ind <= 10: #compare two nucleotide sites if they are close (<10nt)
            if filtered_entropy[idx] >= filtered_entropy[temp_idx]:
                del ind_list[-1]
                ind_list.append(ind)
                del e_list[-1]
                e_list.append(filtered_entropy[idx])
        else: #add new index and entropy to list if it's far away from previously saved data
            ind_list.append(ind)
            e_list.append(filtered_entropy[idx])
        temp_ind = ind
        temp_idx = idx
        temp_e = filtered_entropy[idx]
    ind_array = np.array(ind_list)
    e_array = np.array(e_list)
    return ind_array, e_array



In [10]:
#repeat all of the process for all chromosomes
total_idx_arr = np.array([])
total_entr_arr = np.array([])
chr_mark_list = list()


#chr_list = {'chr11'}
for chr in chr_list:
    chr_cigarDic, chr_seqDic, chr_mapDic, chr_reverseDic = readinSAM(chr)
    chrseq = readinFasta(chr)
    edited_chr_seqDic = matchFrame(chr_cigarDic, chr_seqDic)
    pos_chr_nuc_arr, neg_chr_nuc_arr, pos_chr_depth_arr, neg_chr_depth_arr = seqcompare(chrseq, edited_chr_seqDic, chr_mapDic, chr_reverseDic)   
    pos_entropy_arr = calcShannon(pos_chr_nuc_arr)
    neg_entropy_arr = calcShannon(neg_chr_nuc_arr)
    pos_idx_arr, pos_entr_arr = filter_index_list(pos_entropy_arr, pos_chr_depth_arr)
    neg_idx_arr, neg_entr_arr = filter_index_list(neg_entropy_arr, neg_chr_depth_arr)
    chr_mark_list.extend(['p'+chr]*len(pos_idx_arr))
    chr_mark_list.extend(['n'+chr]*len(neg_idx_arr))
    total_idx_arr = np.append(total_idx_arr, pos_idx_arr)
    total_idx_arr = np.append(total_idx_arr, neg_idx_arr)
    total_entr_arr = np.append(total_entr_arr, pos_entr_arr)
    total_entr_arr = np.append(total_entr_arr, neg_entr_arr)   
    print("end of %s"%chr)
    

2021-07-02 12:33:36.063112
2021-07-02 12:33:53.756122
2021-07-02 12:34:10.694829
end of chr1
2021-07-02 12:35:42.951719
2021-07-02 12:36:00.635008
2021-07-02 12:36:17.803591
end of chr2
2021-07-02 12:37:28.745389
2021-07-02 12:37:45.511649
end of chr3
2021-07-02 12:38:59.071595
2021-07-02 12:39:16.027080
end of chr4
2021-07-02 12:40:29.148226
2021-07-02 12:40:46.460482
end of chr5
2021-07-02 12:42:00.208138
2021-07-02 12:42:17.777245
2021-07-02 12:42:35.193308
end of chr6
2021-07-02 12:43:31.365152
2021-07-02 12:43:48.249737
end of chr7
2021-07-02 12:44:54.667182
2021-07-02 12:45:13.199511
end of chr8
2021-07-02 12:46:21.321153
2021-07-02 12:46:38.549374
end of chr9
2021-07-02 12:47:45.440188
2021-07-02 12:48:02.813876
end of chr10
2021-07-02 12:49:17.811425
2021-07-02 12:49:35.936393
2021-07-02 12:49:53.627579
end of chr11
2021-07-02 12:50:57.134396
2021-07-02 12:51:13.917766
end of chr12
2021-07-02 12:52:10.252982
2021-07-02 12:52:27.718171
end of chr13
2021-07-02 12:53:24.668903
202

In [64]:
len(total_idx_arr)





29150

In [71]:
#rank each positions by entropy and keep only nucleotide positions with the highest 1000 entropies
ind = np.argpartition(total_entr_arr, -1000)[-1000:] #get index for top 1000 entropy in entropy array
complete_pos_arr = np.take(total_idx_arr, ind).astype(int) #keep nucleotide values only in accordance with the previously retrieved index
top_chr_mark_list = list()
for i in ind:
    top_chr_mark_list.append(chr_mark_list[i]) #get chromosome values only in accordance with the previously retrieved index 


In [69]:
#get fasta sequence for each mapped position and write sequence as in a new fasta file
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

def writeFasta (chrNum, chr_pos_list, strand_list):
    for record in SeqIO.parse("./references/GRCm39.primary_assembly.genome.fa", "fasta"):
        if record.id == chrNum:
            for i in range(len(chr_pos_list)):
                records = []
                pos = chr_pos_list[i]
                if strand_list[i] == 'p':
                    records.append(SeqRecord(record.seq[pos-10:pos+11].transcribe()))
                elif strand_list[i] == 'n':
                    records.append(SeqRecord(record.seq[pos-10:pos+11].transcribe().reverse_complement()))
                SeqIO.write(records,fw,"fasta")
                
                
                


In [70]:
import datetime
#run the writeFasta function for each chromosome. in order to do this, go through list of positions with top entropy and separate them into distinct lists of specific chromosomes. 
with open ("./forweblogo.fa","w") as fw:
    print(datetime.datetime.now())
    for chr in chr_list:
        chr_pos_list = []
        strand_list = []
        for i in range(len(top_chr_mark_list)):
            if top_chr_mark_list[i][1:] == chr:
                chr_pos_list.append(complete_pos_arr[i])
                strand_list.append(str(top_chr_mark_list[i])[0])
        if chr_pos_list:  
            writeFasta(chr, chr_pos_list, strand_list)
        print(datetime.datetime.now())
        print('%s end'%chr)
    

2021-07-02 14:43:14.208272
2021-07-02 14:43:29.213786
chr1 end
2021-07-02 14:43:44.242462
chr2 end
2021-07-02 14:43:59.392171
chr3 end
2021-07-02 14:44:14.575880
chr4 end
2021-07-02 14:44:29.808333
chr5 end
2021-07-02 14:44:45.286344
chr6 end
2021-07-02 14:45:00.724582
chr7 end
2021-07-02 14:45:16.026852
chr8 end
2021-07-02 14:45:31.329946
chr9 end
2021-07-02 14:45:46.632082
chr10 end
2021-07-02 14:46:01.892546
chr11 end
2021-07-02 14:46:17.107687
chr12 end
2021-07-02 14:46:32.323997
chr13 end
2021-07-02 14:46:47.492159
chr14 end
2021-07-02 14:47:02.731923
chr15 end
2021-07-02 14:47:17.891189
chr16 end
2021-07-02 14:47:33.061258
chr17 end
2021-07-02 14:47:48.212053
chr18 end
2021-07-02 14:48:03.282882
chr19 end
2021-07-02 14:48:18.408262
chrX end
2021-07-02 14:48:33.570457
chrY end
2021-07-02 14:48:48.771271
chrM end
2021-07-02 14:48:48.772602
GL456210.1 end
2021-07-02 14:48:48.773781
GL456211.1 end
2021-07-02 14:48:48.774959
GL456212.1 end
2021-07-02 14:48:48.776626
GL456219.1 end
202